### ⚠️ Note: NVIDIA Driver Installation Required ⚠️

Verify with: ```nvidia-smi```

In [1]:
from matchmaker import matchmaker

/home/zainansari/miniforge3/envs/matchmaker-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
engine = matchmaker.MatchingEngine()

✅ GPU acceleration enabled


In [3]:
engine.load_interactions("data/swipes.csv", 
  decider_col='decidermemberid', 
  other_col='othermemberid', 
  like_col='like', 
  timestamp_col='timestamp')

Reading data... ✅
Constructing graph...✅
Fitting ALS... 🚀 Starting collaborative filtering training...
📊 Detected cuDF DataFrame, converting to pandas...
📊 Creating user mappings...
📈 49,676 users in square matrix
💾 4,985,185 unique interactions
🔨 Building square sparse matrix...
📊 Matrix density: 0.2020%
🎯 Training ALS model...


100%|██████████| 15/15 [00:00<00:00, 20.07it/s]

🔄 Converting factors to numpy arrays...
✅ Model trained in 1.00s
⏱️ Total processing time: 5.57s
📋 Factor arrays: user_factors (49676, 64), item_factors (49676, 64)
Complete! ✅


In [4]:
user_df = engine.user_df
interaction_df = engine.interaction_df
interaction_graph = engine.interaction_graph
als_model = engine.als_model